In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import joblib
import cv2
import plotly.express as px

from os.path import join
from glob import glob
from PIL import Image
from ipywidgets import interact
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from typing import Callable

In [ ]:
%matplotlib inline

In [ ]:
DATA_FOLDER = 'archive'

In [ ]:
with open(join(DATA_FOLDER, 'cat_to_name.json'), 'r') as fin:
    cat_to_name = json.load(fin)

In [ ]:
COIN_TYPE_I, CURRENCY_I, COUNTRY_I = 0, 1, 2
SEP = ','
cat_to_name = {k: name.split(SEP) for k, name in cat_to_name.items()}
cat_name_df = pd.DataFrame(cat_to_name).transpose()
cat_name_df.rename(columns={0: 'coin type', 1: 'currency', 2: 'country'}, inplace=True)
cat_name_df['class'] = cat_name_df.index

In [ ]:
train_data_pathes = glob(join(DATA_FOLDER, 'coins', 'data', 'train', '*', '*'))
test_data_pathes = glob(join(DATA_FOLDER, 'coins', 'data', 'test', '*', '*'))
validation_data_pathes = glob(join(DATA_FOLDER, 'coins', 'data', 'validation', '*', '*'))

In [ ]:
def load_coin_data(pathes: list[str]) -> dict[str, np.ndarray]:
    CLASS_PATH_I = -2
    FILE_PATH_I = -1
    FILE_IDX_SEP = '__'
    data = dict()
    
    for path in pathes:
        coin_class = path.split(os.sep)[CLASS_PATH_I]
        coin_idx = path.split(os.sep)[FILE_PATH_I].split(FILE_IDX_SEP)[0]
        data[f'{coin_class}_{coin_idx}'] = tf.convert_to_tensor(
            tf.keras.utils.load_img(path), dtype_hint=tf.float32
        )

    return data

In [ ]:
train_data = load_coin_data(train_data_pathes)

## Detect images with front and back

In [ ]:
IMG_REL_THRESHOLD = 1.35
train_img_rel = {
    k: img.shape[1] / img.shape[0]
    for k, img in train_data.items()
}
front_back_keys = [
    k for k in train_img_rel.keys()
    if train_img_rel[k] >= IMG_REL_THRESHOLD
]
front_back_map = {
    class_val: [k for k in front_back_keys if k.split('_')[0] == class_val]
    for class_val in cat_to_name.keys()
}
cat_name_df['front_back_keys'] = cat_name_df['class'].map(front_back_map)

### It is needed to detect front class and back class

In [ ]:
@tf.function
def make_sobel_image(img_tensor: tf.Tensor) -> tf.Tensor:
    if len(img_tensor.shape) == 3:
        grad_sobel = tf.image.sobel_edges(tf.expand_dims(img_tensor, axis=0))
    elif len(img_tensor.shape) == 4:
        grad_sobel = tf.image.sobel_edges(img_tensor)
    else:
        raise AttributeError("Invalid image tensor shape, should be either 3 or 4")

    grad_module = tf.sqrt(tf.reduce_sum(grad_sobel ** 2, axis=-1))
    return tf.reshape(grad_module, img_tensor.shape)

In [ ]:
FORCE_LOAD = False
IS_SAVE = False

SAVE_PATH = join(DATA_FOLDER, 'train_edges_imgs.joblib')

if not os.path.exists(SAVE_PATH) or FORCE_LOAD:
    print('preprocessing...')
    train_edge_imgs = {k: make_sobel_image(img)[:, :, 0] for k, img in train_data.items()}
else:
    print('loading from file...')
    with open(SAVE_PATH, 'rb') as fin:
        train_edge_imgs = joblib.load(fin)

if IS_SAVE:
    with open(SAVE_PATH, 'wb') as fout:
        joblib.dump(train_edge_imgs, fout)

deleting the ucoin.net template

In [ ]:
ucoin_text_templates_vertical = [
    train_edge_imgs['1_001'][-408:-310, -22:].numpy(),
    train_edge_imgs['1_007'][72:141, -15:].numpy()
]
ucoin_text_templates_horizontal = [
    cv2.rotate(template, cv2.ROTATE_90_CLOCKWISE)
    for template in ucoin_text_templates_vertical
]

In [ ]:
@interact
def view_template_detection(input_k=train_edge_imgs.keys()):
    global ucoin_text_templates_horizontal
    global ucoin_text_templates_vertical
    
    input_img = train_edge_imgs[input_k].numpy()
    CORR_THRESHOLD = 0.9
    detected_vert_indices = []
    detected_horiz_indices = []
    for horiz_template, vert_template in zip(
        ucoin_text_templates_horizontal,
        ucoin_text_templates_vertical
    ):
        vert_corr = cv2.matchTemplate(input_img, vert_template, cv2.TM_CCOEFF_NORMED)
        horiz_corr = cv2.matchTemplate(input_img, horiz_template, cv2.TM_CCOEFF_NORMED)
        detected_vert_indices.append(np.where(vert_corr >= CORR_THRESHOLD))
        detected_horiz_indices.append(np.where(horiz_corr >= CORR_THRESHOLD))

    print('detected vertical indices', detected_vert_indices)
    print('detected horizontal indices', detected_horiz_indices)
    
    for i in range(len(detected_horiz_indices)):
        for pt in zip(*detected_vert_indices[i]):
            cv2.rectangle(
                input_img,
                (pt[1], pt[0]),
                (pt[1] + ucoin_text_templates_vertical[i].shape[1],
                 pt[0] + ucoin_text_templates_vertical[i].shape[0]),
                255,
                2
            )
        for pt in zip(*detected_horiz_indices[i]):
            cv2.rectangle(
                input_img,
                (pt[1], pt[0]),
                (pt[1] + ucoin_text_templates_horizontal[i].shape[1],
                 pt[0] + ucoin_text_templates_horizontal[i].shape[0]),
                255,
                2
            )
    
    plt.imshow(input_img)

### crop images (delete vertical ucoin.net band)

In [ ]:
FORCE_LOAD = False
IS_SAVE = True
SAVE_PATH = join(DATA_FOLDER, 'ucoin_cropped_train_edges_imgs.joblib')

cropped_edge_imgs = dict()
CORR_THRESHOLD = 0.9

if not os.path.exists(SAVE_PATH) or FORCE_LOAD: 
    for key in train_edge_imgs:
        input_img = train_edge_imgs[key].numpy()
        detected_vert_indices = []
    
        for vert_template in ucoin_text_templates_vertical:
            vert_corr = cv2.matchTemplate(input_img, vert_template, cv2.TM_CCOEFF_NORMED)
            detected_vert_indices.append(np.where(vert_corr >= CORR_THRESHOLD))
    
        template_xs = []
        for i in range(len(detected_vert_indices)):
            for pt in zip(*detected_vert_indices[i]):
                template_xs.append(pt[1])
        
        if len(template_xs) != 0:
            x_band_mean = int(np.mean(template_xs))
        
            # the algorithm of determing the position of the band
            # (left side/right side)
            if x_band_mean > (input_img.shape[1] / 2.0):
                cropped_edge_imgs[key] = input_img[:, :x_band_mean]
            else:
                cropped_edge_imgs[key] = input_img[:, x_band_mean:]
        else:
            cropped_edge_imgs[key] = input_img
else:
    print('loading from file...')
    with open(SAVE_PATH, 'rb') as fin:
        cropped_edge_imgs = joblib.load(fin)

if IS_SAVE:
    print('saving results...')
    with open(SAVE_PATH, 'wb') as fout:
        joblib.dump(cropped_edge_imgs, fout)

In [ ]:
# @interact
# def view_imgs(input_key=list(cropped_edge_imgs.keys())):
#     plt.imshow(cropped_edge_imgs[input_key])

### using Embedding algorithms to cluster by two images (front and back)

In [ ]:
def make_badge_scatter(
    df: pd.DataFrame,
    img_provider:Callable[[str], OffsetImage],
    **subplot_kwargs
) -> tuple:
    REQ_COLS = {'x', 'y', 'keys'}
    if len(set(df.columns).union(REQ_COLS)) > len(df.columns):
        raise ValueError(f'Required cols for the input dataframe {REQ_COLS}')
        
    fig, ax = plt.subplots(**subplot_kwargs)
    ax.scatter(df['x'], df['y'])
    for index, row in df.iterrows():
        ab = AnnotationBbox(getImage(row['keys']), (row['x'], row['y']),
                            frameon=False)
        ax.add_artist(ab)
    plt.grid(True)
    return fig, ax

In [ ]:
def getImage(key):
    return OffsetImage(cropped_edge_imgs[key], zoom=0.25, alpha=1)


@interact
def view_embed(input_class=cat_name_df['class'].to_list()):
    IMG_SIZE = [100, 100]
    embed_m = TSNE(n_components=2, perplexity=5)
    
    input_flatten_imgs = {
        key: np.expand_dims(
            cv2.resize(img, IMG_SIZE).astype(np.int32).flatten(),
            axis=0
        )
        for key, img in cropped_edge_imgs.items()
        if key.split('_')[0] == input_class 
            and key not in cat_name_df[cat_name_df['class'] == input_class]\
                           .iloc[0]['front_back_keys']
    }
    
    iter_keys = sorted(input_flatten_imgs.keys())
    data_to_embed = np.concatenate([input_flatten_imgs[k] for k in iter_keys], axis=0)
    
    res = pd.DataFrame(embed_m.fit_transform(data_to_embed), columns=['x', 'y'])
    res['keys'] = iter_keys
    fig, ax = make_badge_scatter(res, getImage, figsize=(20, 20))
    fig.show()

- 2 class has 1 element of the 1 class
- 15 class has 1 element of the 16 class

TODO: using IIC to cluster front/back